In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [10]:
api_wrapper = WikipediaAPIWrapper(top_k_results = 1, doc_content_chars_max=200)
wiki_tool = WikipediaAPIWrapper(api_wrapper=api_wrapper)

In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()




In [15]:
from langchain.tools.retriever import create_retriever_tool

retrieve_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about langsmith. For any questions about Langsmith, you should use this tool")

In [16]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max = 200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [23]:
tools = [arxiv, retrieve_tool]

In [20]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [21]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
"""
Agents use language models to choose sequence of actions, while in chains sequence is pre defined, agents use language models for reasoning. Here agent will use the tools to search for information from different sources 
"""
# Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)



In [25]:
# Agent Executor
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [26]:
agent_executor.invoke({"input": "Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | ğŸ¦œï¸�ğŸ› ï¸� LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!
1. Install LangSmithâ€‹
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API keyâ€‹
To create an API key head to the Settings page. Then click Create API Key.
3. Set up your environmentâ€‹
Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though

{'input': 'Tell me about Langsmith',
 'output': "Langsmith is a platform designed for building production-grade applications using large language models (LLMs). It emphasizes the ability to closely monitor and evaluate applications, enabling developers to deploy their solutions quickly and confidently. Here are some key features and steps to get started with Langsmith:\n\n1. **Installation**: You can install Langsmith using Python or TypeScript. For Python, you can use the command:\n   ```bash\n   pip install -U langsmith openai\n   ```\n   For TypeScript, you can use:\n   ```bash\n   yarn add langsmith openai\n   ```\n\n2. **API Key Creation**: To use Langsmith, you need to create an API key. This can be done from the Settings page of the platform.\n\n3. **Environment Setup**: You need to set up your environment variables, including the LangChain tracing and API keys.\n\n4. **Logging Traces**: After setting up, you can log your first trace and access the LangSmith experiments UI to ex

In [29]:
agent_executor.invoke({"input": "state of the art llm models"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'state of the art LLM models'}`


Published: 2024-05-30
Title: A Survey Study on the State of the Art of Programming Exercise Generation using Large Language Models
Authors: Eduard Frankford, Ingo Höhn, Clemens Sauerwein, Ruth Breu
Su
Invoking: `arxiv` with `{'query': 'large language models'}`


Published: 2023-06-12
Title: Lost in Translation: Large Language Models in Non-English Content Analysis
Authors: Gabriel Nicholas, Aliya Bhatia
Summary: In recent years, large language models (e.g., O
Invoking: `arxiv` with `{'query': 'latest advancements in large language models'}`


Published: 2024-04-13
Title: Navigating the Landscape of Large Language Models: A Comprehensive Review and Analysis of Paradigms and Fine-Tuning Strategies
Authors: Benjue Weng
Summary: With the surge
Invoking: `langsmith_search` with `{'query': 'state of the art LLM models'}`


Get started with LangSmith | ğŸ¦œï¸�ğŸ› ï¸� LangSmith

Skip to

{'input': 'state of the art llm models',
 'output': 'Here are some recent insights and publications regarding the state-of-the-art in large language models (LLMs):\n\n1. **Survey on Programming Exercise Generation**:\n   - **Title**: A Survey Study on the State of the Art of Programming Exercise Generation using Large Language Models\n   - **Authors**: Eduard Frankford, Ingo Höhn, Clemens Sauerwein, Ruth Breu\n   - **Published**: May 30, 2024\n   - This paper surveys the advancements in using LLMs for generating programming exercises, highlighting the capabilities and methodologies employed in this domain.\n\n2. **Non-English Content Analysis**:\n   - **Title**: Lost in Translation: Large Language Models in Non-English Content Analysis\n   - **Authors**: Gabriel Nicholas, Aliya Bhatia\n   - **Published**: June 12, 2023\n   - This work discusses the application of LLMs in analyzing non-English content, addressing challenges and opportunities in multilingual contexts.\n\n3. **Comprehensi